# Criptografia Homomórfica com Paillier

1. [Introdução à Criptografia Homomórfica](#introdução-à-criptografia-homomórfica)
1. [O esquema](#o-esquema-paillier)
1. [Implementação](#implementação-do-esquema-paillier)
1. [Oprações Homomórficas](#operações-homomórficas)
1. [Aplicações](#aplicações)


## Introdução à Criptografia Homomórfica

A **criptografia homomórfica** é um tipo especial de criptografia que permite realizar operações matemáticas diretamente sobre dados criptografados, sem a necessidade de descriptografá-los primeiro.

### 1.1 Propriedade Homomórfica

Para um esquema de criptografia ser homomórfico, ele deve satisfazer:

$$E(m_1 \oplus m_2) = E(m_1) \odot E(m_2)$$

Onde:
- $E()$ é a função de criptografia
- $m_1, m_2$ são mensagens (dados originais)
- $\oplus$ é uma operação no espaço das mensagens
- $\odot$ é uma operação no espaço dos dados criptografados

### 1.2 Tipos de Homomorfismo

1. **Homomorfismo Aditivo**: $E(m_1 + m_2) = E(m_1) \cdot E(m_2)$
2. **Homomorfismo Multiplicativo**: $E(m_1 \cdot m_2) = E(m_1) \otimes E(m_2)$
3. **Homomorfismo Total (FHE)**: Suporta ambas as operações

## O Esquema Paillier



O esquema **Paillier** é um sistema de criptografia assimétrica que possui propriedades homomórficas **aditivas**.

### 2.1 Propriedades Matemáticas

O esquema Paillier satisfaz:

$$E(m_1 + m_2) = E(m_1) \times E(m_2) \bmod n^2$$

$$E(k \cdot m) = E(m)^k \bmod n^2$$

### 2.2 Parâmetros do Sistema

- **Chave Pública**: $(n, g)$ onde $n = p \cdot q$ e $g \in \mathbb{Z}_{n^2}^*$
- **Chave Privada**: $(\lambda, \mu)$ onde:
  - $\lambda = \text{lcm}(p-1, q-1)$
  - $\mu = (L(g^\lambda \bmod n^2))^{-1} \bmod n$
  - $L(x) = \frac{x-1}{n}$

### 2.3 Algoritmos

**Criptografia:**
$$c = g^m \cdot r^n \bmod n^2$$

**Descriptografia:**
$$m = L(c^\lambda \bmod n^2) \cdot \mu \bmod n$$

Onde $r$ é um número aleatório tal que $\gcd(r,n) = 1$.

## Implementação do Esquema Paillier

In [ ]:
import random
import math
from typing import Tuple
import numpy as np


### Funções auxiliares

In [ ]:
    def _generate_prime(self, bits: int) -> int:
        """Gera um número primo com o número especificado de bits"""
        while True:
            # Gerar número ímpar aleatório
            num = random.getrandbits(bits)
            if num % 2 == 0:
                num += 1
            if self._is_prime(num):
                return num
    
    def _is_prime(self, n: int, k: int = 5) -> bool:
        """Teste de primalidade de Miller-Rabin"""
        if n < 2:
            return False
        if n == 2 or n == 3:
            return True
        if n % 2 == 0:
            return False
        
        # Escrever n-1 como d × 2^r
        r = 0
        d = n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        
        # Realizar k testes
        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, d, n)
            
            if x == 1 or x == n - 1:
                continue
            
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        
        return True
    
    def _lcm(self, a: int, b: int) -> int:
        """Calcula o menor múltiplo comum"""
        return abs(a * b) // math.gcd(a, b)
    
    def _mod_inverse(self, a: int, m: int) -> int:
        """Calcula o inverso modular usando algoritmo estendido de Euclides"""
        if math.gcd(a, m) != 1:
            raise ValueError("Inverso modular não existe")
        
        def extended_gcd(a, b):
            if a == 0:
                return b, 0, 1
            gcd, x1, y1 = extended_gcd(b % a, a)
            x = y1 - (b // a) * x1
            y = x1
            return gcd, x, y
        
        _, x, _ = extended_gcd(a % m, m)
        return (x % m + m) % m

In [ ]:
##
## Criptografar
##
def encrypt(self, message: int) -> int:
    """
    Criptografa uma mensagem usando a chave pública
    
    Implementa: c = g^m × r^n mod n²
    
    Args:
        message: Mensagem a ser criptografada (inteiro não-negativo)
        
    Returns:
        Texto cifrado
    """
    n, g = self.public_key
    n_squared = n * n
    
    # Validar entrada
    if message < 0 or message >= n:
        raise ValueError(f"Mensagem deve estar no intervalo [0, {n-1}]")
    
    # Escolher r aleatório onde gcd(r, n) = 1
    while True:
        r = random.randrange(1, n)
        if math.gcd(r, n) == 1:
            break
    
    # Calcular c = g^m × r^n mod n²
    ciphertext = (pow(g, message, n_squared) * pow(r, n, n_squared)) % n_squared
    return ciphertext

def decrypt(self, ciphertext: int) -> int:
    """
    Descriptografa um texto cifrado usando a chave privada
    
    Implementa: m = L(c^λ mod n²) × μ mod n
    onde L(x) = (x-1)/n
    
    Args:
        ciphertext: Texto cifrado a ser descriptografado
        
    Returns:
        Mensagem original
    """
    lambda_n, mu, n = self.private_key
    n_squared = n * n
    
    # Calcular c^λ mod n²
    c_lambda = pow(ciphertext, lambda_n, n_squared)
    
    # Aplicar função L(x) = (x-1)/n
    l_val = (c_lambda - 1) // n
    
    # Calcular m = L(c^λ mod n²) × μ mod n
    message = (l_val * mu) % n
    
    return message

### Crypto Sistema Paillier

In [ ]:
class PaillierCryptosystem:
    """
    Implementação do sistema criptográfico Paillier
    
    O esquema Paillier é baseado no problema da residuosidade quadrática composta
    e oferece homomorfismo aditivo.
    """
    
    def __init__(self, key_size: int = 512):
        """
        Inicializa o sistema criptográfico Paillier
        
        Args:
            key_size: Tamanho da chave em bits
        """
        self.key_size = key_size
        self.public_key, self.private_key = self._generate_keys()
        
    def _generate_keys(self) -> Tuple[Tuple[int, int], Tuple[int, int, int]]:
        """
        Gera as chaves pública e privada do esquema Paillier
        
        Returns:
            Tupla contendo (chave_publica, chave_privada)
            - chave_publica: (n, g)
            - chave_privada: (lambda_n, mu, n)
        """
        # Passo 1: Gerar dois números primos p e q de tamanho similar
        p = self._generate_prime(self.key_size // 2)
        q = self._generate_prime(self.key_size // 2)
        
        # Passo 2: Calcular n = p × q
        n = p * q
        
        # Passo 3: Calcular λ = lcm(p-1, q-1)
        lambda_n = self._lcm(p - 1, q - 1)
        
        # Passo 4: Escolher g = n + 1 (forma mais simples e segura)
        g = n + 1
        
        # Passo 5: Calcular μ = (L(g^λ mod n²))^(-1) mod n
        n_squared = n * n
        gcd_val = pow(g, lambda_n, n_squared)
        l_val = (gcd_val - 1) // n  # Função L(x) = (x-1)/n
        mu = self._mod_inverse(l_val, n)
        
        # Retornar chaves
        public_key = (n, g)
        private_key = (lambda_n, mu, n)
        
        return public_key, private_key
    
# add métodos
PaillierCryptosystem._generate_prime = _generate_prime
PaillierCryptosystem._is_prime = _is_prime
PaillierCryptosystem._lcm = _lcm
PaillierCryptosystem._mod_inverse = _mod_inverse

PaillierCryptosystem.encrypt = encrypt
PaillierCryptosystem.decrypt = decrypt

## Operações Homomórficas



### 4.1 Soma Homomórfica

A propriedade fundamental do Paillier é:

$$E(m_1 + m_2) = E(m_1) \times E(m_2) \bmod n^2$$

Isso significa que podemos somar dois números criptografados multiplicando seus valores cifrados!



In [ ]:
def add_encrypted(self, c1: int, c2: int) -> int:
    """
    Soma dois valores criptografados
    
    Implementa: E(m1 + m2) = E(m1) × E(m2) mod n²
    
    Args:
        c1, c2: Valores criptografados a serem somados
        
    Returns:
        E(m1 + m2): Resultado da soma (criptografado)
    """
    n, _ = self.public_key
    n_squared = n * n
    
    return (c1 * c2) % n_squared

PaillierCryptosystem.add_encrypted = add_encrypted

### 4.2 Multiplicação por Escalar

Também podemos multiplicar um valor criptografado por uma constante:

$$E(k \times m) = E(m)^k \bmod n^2$$

In [ ]:

def multiply_by_constant(self, ciphertext: int, constant: int) -> int:
    """
    Multiplica um valor criptografado por uma constante
    
    Implementa: E(k × m) = E(m)^k mod n²
    
    Args:
        ciphertext: Valor criptografado
        constant: Constante para multiplicação
        
    Returns:
        E(k × m): Resultado da multiplicação (criptografado)
    """
    n, _ = self.public_key
    n_squared = n * n
    
    if constant < 0:
        raise ValueError("Constante deve ser não-negativa")
    
    return pow(ciphertext, constant, n_squared)

PaillierCryptosystem.multiply_by_constant = multiply_by_constant


In [ ]:
def get_public_key_info(self):
    """Retorna informações sobre a chave pública"""
    n, g = self.public_key
    return {
        'n': n,
        'g': g,
        'n_bits': n.bit_length(),
        'max_message': n - 1
    }

PaillierCryptosystem.get_public_key_info = get_public_key_info

## Demonstração

Vamos agora demonstrar as propriedades homomórficas do esquema Paillier com exemplos práticos.

In [ ]:
paillier = PaillierCryptosystem(key_size=256)  # Chave menor para demonstração rápida

# Mostrar informações da chave pública
key_info = paillier.get_public_key_info()
print(f"\nInformações da Chave Pública:")
print(f"n = {key_info['n']}")
print(f"g = {key_info['g']}")
print(f"Tamanho de n: {key_info['n_bits']} bits")
print(f"Mensagem máxima: {key_info['max_message']}")

### 5.1 Teste da Criptografia Básica

In [ ]:
# Valores de teste
messages = [42, 100, 255, 1000]

for msg in messages:
    # Criptografar
    encrypted = paillier.encrypt(msg)
    
    # Descriptografar
    decrypted = paillier.decrypt(encrypted)
    
    # Verificar
    success = "✓" if decrypted == msg else "✗"
    print(f"{success} m = {msg:4d} | E(m) = ... | D(E(m)) = {decrypted:4d}")


O que acontece se colocarmos umas msg como `-1`?

### 5.2 Demonstração da Soma Homomórfica

Vamos demonstrar que $E(m_1 + m_2) = E(m_1) \times E(m_2) \bmod n^2$

In [ ]:
# Valores para teste
m1, m2 = 15, 25
print(f"Soma esperada: m₁ + m₂ = {m1 + m2}\n")

In [ ]:
# Criptografar os valores
c1 = paillier.encrypt(m1)
c2 = paillier.encrypt(m2)

# Realizar a soma homomórfica
homomorphic_sum = paillier.add_encrypted(c1, c2)

c1,c2

In [ ]:
# Descriptografar o resultado
print(f"Resultado da soma homomórfica (cifrado): {homomorphic_sum}")

In [ ]:
decrypted_homomorphic = paillier.decrypt(homomorphic_sum)
print(f"Resultado da soma homomórfica (descriptografado): {decrypted_homomorphic}")

### 5.3 Demonstração da Multiplicação por Escalar

Vamos demonstrar que $E(k \times m) = E(m)^k \bmod n^2$

In [ ]:
m = 12
k = 7

print(f"Produto esperado: k × m = {k} × {m} = {k * m}\n")


In [ ]:
# Criptografar o valor
c = paillier.encrypt(m)

# Realizar a multiplicação homomórfica
homomorphic_product = paillier.multiply_by_constant(c, k)
homomorphic_product

In [ ]:

# Descriptografar o resultado
decrypted_homomorphic = paillier.decrypt(homomorphic_product)
decrypted_homomorphic


### 5.4 Operações Complexas

Vamos demonstrar operações mais complexas combinando as propriedades homomórficas:

Calcularemos $k \times (m_1 + m_2)$ de forma homomórfica através de $E(k \times (m_1 + m_2)) = (E(m_1) \times E(m_2))^k \bmod n^2$

In [ ]:
# Valores para teste
m1, m2, k = 8, 12, 5
expected = k * (m1 + m2)
print(f"Resultado esperado: k × (m₁ + m₂) = {k} × ({m1} + {m2}) = {expected}\n")

In [ ]:
# Criptografar m1 e m2
c1 = paillier.encrypt(m1)
c2 = paillier.encrypt(m2)

# Passo 1: Soma homomórfica de c1 e c2 para obter E(m1 + m2)
c_sum = paillier.add_encrypted(c1, c2)

# Passo 2: Multiplicação por escalar de E(m1 + m2) por k
c_final = paillier.multiply_by_constant(c_sum, k)

In [ ]:
# Descriptografar o resultado final
decrypted_final = paillier.decrypt(c_final)

print(f"Resultado da operação complexa (descriptografado): {decrypted_final}")


## Aplicações

O paillier é  útil em cenários que exigem a agregação de dados privados, como:
   - **Votação Eletrônica (e-voting)**: Contar votos sem revelar as escolhas individuais.
   - **Leilões Privados**: Determinar o vencedor sem revelar os lances dos outros participantes.
   - **Análise de Dados Médicos**: Calcular estatísticas (como médias) de dados de pacientes de diferentes hospitais sem que os hospitais compartilhem os dados brutos.

Embora não seja **totalmente homomórfico** (não suporta multiplicação entre dois números criptografados), suas propriedades são poderosas para muitas aplicações práticas que preservam a privacidade.

### Eleições Seguras: Preservando a Privacidade do processo eleitoral

Na tranquila universidade UFCG, a comunidade acadêmica demonstrava crescente preocupação com a transparência e a privacidade no processo de escolha do novo reitor. A eleição entre os candidatos Alice e Bob precisava ser absolutamente segura, garantindo a inviolabilidade dos votos.

No entanto, ao examinar o código do sistema de votação, um estudante do curso de computação fez uma descoberta alarmante: a urna eletrônica armazenava os votos de forma desprotegida, permitindo que um agente malicioso pudesse realizar a reidentificação dos eleitores — comprometendo a confidencialidade do processo eleitoral.

segue o trecho da código da urna:

- Onde 1 indicava votos em Alice e 0 em Bob.

In [ ]:
# Votos: 1 para candidato A, 0 para candidato B
votos = [1, 0, 1, 1, 0, 1, 0, 1, 0, 1]  # 6 votos para A, 4 para B
print(f"Votos originais: {votos}")
print(f"Candidato A: {sum(votos)} votos")
print(f"Candidato B: {len(votos) - sum(votos)} votos")

A comissão eleitoral especificou um sistema revolucionário de votação eletrônica seguro , que garante:

- **Privacidade Individual**: Nenhum voto pode ser identificado
- **Integridade dos Resultados**: Contagem precisa e segura
- **Transparência**: Possibilidade de verificação do resultado final

#### Funcionanamento

1. **Registro do Voto**
   - Cada eleitor vota digitalmente
   - Voto é imediatamente criptografado
   - Valor 1 representa voto no Candidato A
   - Valor 0 representa voto no Candidato B

2. **Processo de Criptografia**
   - Cada voto é transformado em um código secreto
   - Impossível descobrir o voto original
   - Apenas o resultado final pode ser descriptografado

3. **Contagem Segura**
   - Votos criptografados são computados
   - Resultado final é descriptografado
   - Preserva o anonimato de cada eleitor

#### Passo 1: criptografar os dados

In [ ]:
# Criptografar cada voto
votos_criptografados = [paillier.encrypt(voto) for voto in votos]

len(votos_criptografados)

In [ ]:
votos_criptografados

#### Passo 2: Soma dos votos

In [ ]:
# Contar votos de forma segura (somando todos os valores criptografados)
total_votos_A = votos_criptografados[0]
for i in range(1, len(votos_criptografados)):
    total_votos_A = paillier.add_encrypted(total_votos_A, votos_criptografados[i])

In [ ]:
total_votos_A

#### Passo 3: Como computar os votos de cada cantidato?

In [ ]:
# Descriptografar apenas o resultado final
resultado_A = paillier.decrypt(total_votos_A)
resultado_B = len(votos) - resultado_A

print(f"Resultado da eleição:")
print(f"\t Candidato Alice: {resultado_A} votos")
print(f"\t Candidato Bob: {resultado_B} votos")


🔒 **Processo de Votação** 🔒
- Votos são criptografados individualmente
- Sistema soma os votos criptografados
- Apenas o resultado final é revelado

### Benefícios

- ✅ Proteção contra fraudes
- ✅ Sigilo do voto garantido
- ✅ Contagem instantânea e precisa
- ✅ Auditoria simplificada